# Building an AI Agent from Scratch

### Install required libraries

In [7]:
!pip install -q openai sentencepiece transformers

### Load the environment variables

In [9]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
os.environ.get('OPENAI_API_KEY')
print("API Key Loaded", os.environ.get('OPENAI_API_KEY') is not None)

API Key Loaded True


### Import Necessary Modules

In [33]:
import openai
import re
import os
import requests
from datetime import datetime, timedelta

### Testing our Configuration

In [11]:
from openai import OpenAI
client = OpenAI()
MODEL='gpt-4o-mini'
prompt = 'Write something short but funny.'
chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            'role':'user', 'content':prompt
        }
    ]
)
chat_completion.choices[0].message.content

'Why did the scarecrow win an award? Because he was outstanding in his field!'

### Creating the Agent Class

In [14]:
class TravelAgent:
    def __init__(self, system_prompt, initial_budget=1000):
        self.system_prompt = system_prompt
        self.budget = initial_budget
        self.messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'system', 'content': f'The current budget is ${self.budget}.'}
        ]
        self.memory = []

    def __call__(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        response = self.execute()
        self.messages.append({"role": "assistant", "content": response})
        return response

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.messages,
            temperature=0.7
        )
        return completion.choices[0].message.content

    def add_to_memory(self, action, result):
        self.memory.append(f"Action: {action}, Result: {result}")

    def set_budget(self, amount):
        self.budget = amount
        self.messages.append({'role': 'system', 'content': f'The current budget is ${self.budget}.'})

    def get_budget(self):
        return self.budget

# Tools for the agent

### Currency Converter Tool

In [18]:
def convert_currency(amount, from_currency, to_currency):
    url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response = requests.get(url)
    data = response.json()
    rate = data['rates'][to_currency]
    converted_amount = amount * rate
    return round(converted_amount, 2)

In [37]:
convert_currency(100, "USD", "EUR")

90.8

### Weather Tool

In [40]:
def get_weather(location: str, days_ahead: int = 0) -> str:
    """
    This function uses the OpenWeatherMap API to get the current weather for a given location.
    """ 

    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    if not api_key:
        raise Exception("OPENWEATHERMAP_API_KEY environment variable not set.")

    if days_ahead > 0:
        target_date = (datetime.now() + timedelta(days=days_ahead)).strftime('%Y-%m-%d')
    else:
        target_date = None

    base_url = "http://api.openweathermap.org/data/2.5/weather" 

    params = {
        "q": location,
        "appid": api_key,
        "units": "metric"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        if target_date:
            return f"The weather in {data['name']}, {data['sys']['country']} on {target_date} is currently {data['weather'][0]['description']} with a temperature of {data['main']['temp']}°C. The humidity is {data['main']['humidity']}% and the wind speed is {data['wind']['speed']} m/s."
        else:
            return f"The weather in {data['name']}, {data['sys']['country']} is currently {data['weather'][0]['description']} with a temperature of {data['main']['temp']}°C. The humidity is {data['main']['humidity']}% and the wind speed is {data['wind']['speed']} m/s."
    else:
        raise Exception(f"Error fetching weather data: {response.status_code}")

In [42]:
get_weather('Dubai',3)

'The weather in Dubai, AE on 2024-09-15 is currently clear sky with a temperature of 44.23°C. The humidity is 18% and the wind speed is 5.14 m/s.'

### Translation Tool

In [45]:
from transformers import pipeline

def translate_text(text, target_language):
    try:
        translator = pipeline("translation", model="facebook/m2m100_418M")
        translation = translator(text, src_lang="en", tgt_lang=target_language)[0]['translation_text']
        return translation
    except Exception as e:
        print(f"An error occurred during translation: {e}")
        return "Translation failed."

In [47]:
english_text = "Hello, where is the nearest resturant?"
target_language = "fr"

translated_text = translate_text(english_text, target_language)
print(f"Original: {english_text}")
print(f"Translated: {translated_text}")

Original: Hello, where is the nearest resturant?
Translated: Où se trouve le restaurant le plus proche ?


# Prompt

In [50]:
travel_agent_prompt = """
You are a Travel Planning Assistant. Your role is to help users plan their trips by providing information about destinations, 
currency conversion, weather forecasts, and language translation. 

You run in a loop of Thought, Action, PAUSE, Observation.
Use Thought to describe your reasoning about the user's request.
Use Action to perform one of the available actions, then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

1. convert_currency:
   e.g., convert_currency: 100, USD, EUR
   Converts the given amount from one currency to another.

2. get_weather:
   e.g., get_weather: Paris, 3
   Gets the weather forecast for the specified city and number of days ahead (0 for today).

3. translate_text:
   e.g., translate_text: Hello, how are you?, fr
   Translates the given text to the specified target language (use language codes like 'fr' for French).

Always consider the user's budget when making suggestions. The current budget will be provided in the system messages.

Example session:

User: I'm planning a trip to Paris next week. What should I know?

Thought: I should check the weather in Paris for next week and provide some basic French phrases.
Action: get_weather: Paris, 7
PAUSE

Observation: Partly cloudy, 22°C

Thought: Now that I have the weather information, I should provide a simple French phrase and its translation.
Action: translate_text: Hello, how are you?, fr
PAUSE

Observation: Bonjour, comment allez-vous ?

Answer: Great! For your trip to Paris next week, here's what you should know:

1. Weather: It's expected to be partly cloudy with a temperature of 22°C (72°F). 
   Pack accordingly with light layers and maybe a light jacket for evenings.

2. Language: While many people in Paris speak some English, it's always appreciated if you try some basic French. 
   Here's a useful phrase: "Hello, how are you?" in French is "Bonjour, comment allez-vous?"

Remember to consider the budget for your trip and let me know if you need any currency conversion or have any other questions
about your Paris adventure!
"""

# Building the AI Travel Agent

### Dictionary of available actions

In [55]:
known_actions = {
    'convert_currency': convert_currency,
    'get_weather': get_weather,
    'translate_text': translate_text
}

### AI Travel Agent

In [57]:
def plan_trip(question, max_turns=5):
    agent = TravelAgent(travel_agent_prompt)
    
    budget_match = re.search(r'\$(\d+)', question)
    if budget_match:
        budget = int(budget_match.group(1))
        agent.set_budget(budget)
    
    next_prompt = question
    
    action_re = re.compile(r'^Action: (\w+): (.+)$')
    
    for i in range(max_turns):
        result = agent(next_prompt)
        print(result)
        
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        
        if actions:
            action, action_input = actions[0].groups()
            
            if action not in known_actions:
                print(f'Unknown action: {action}: {action_input}')
                continue
            
            print(f' -- running {action}: {action_input}')
            
            try:
                if action == 'convert_currency':
                    amount, from_currency, to_currency = action_input.split(',')
                    observation = known_actions[action](float(amount), from_currency.strip(), to_currency.strip())
                elif action == 'get_weather':
                    location, *days = action_input.split(',')
                    days_ahead = int(days[0]) if days else 0
                    observation = known_actions[action](location.strip(), days_ahead)
                elif action == 'translate_text':
                    if ',' in action_input:
                        text, target_lang = action_input.rsplit(',', 1)
                    else:
                        text, target_lang = action_input.rsplit(None, 1)
                    observation = known_actions[action](text.strip(), target_lang.strip())
                
                print(f'Observation: {observation}')
                agent.add_to_memory(action, observation)
                next_prompt = f'Observation: {observation}'
            except Exception as e:
                print(f"Error executing action: {e}")
                next_prompt = f"Error: Unable to execute action {action}. Please try again."
        else:
            return

In [59]:
question = "I'm planning a 5-day trip to Dubai next month. My budget is $2000. What should I prepare?"
plan_trip(question)

Thought: Since the user is planning a trip to Dubai next month, I should check the weather forecast for that time and provide some useful Arabic phrases. Additionally, I should consider the budget for activities and any currency conversion if needed. 

Action: get_weather: Dubai, 30
PAUSE
 -- running get_weather: Dubai, 30
Observation: The weather in Dubai, AE on 2024-10-12 is currently clear sky with a temperature of 39.96°C. The humidity is 42% and the wind speed is 6.17 m/s.
Thought: The weather in Dubai next month is expected to be quite hot, with temperatures around 40°C (104°F). It's essential to prepare for the heat, and I should also provide a simple Arabic phrase for the user to use during their trip.

Action: translate_text: Hello, how are you?, ar
PAUSE
 -- running translate_text: Hello, how are you?, ar
Observation: مرحبا، كيف حالك؟
Answer: Great! For your trip to Dubai next month, here's what you should prepare:

1. **Weather**: Expect clear skies and very hot temperatures